## Урок 4. Домашнее задание

1. Перенесите метрики в модуль metrics.py (убедится что они там)
2. Перенесите функцию prefilter_items в модуль utils.py
3. Создайте модуль recommenders.py. Напищите код для класса ниже 
(задание обсуждали на вебинаре, для первой функции практически сделали) и положите его в recommenders.py
4. Проверьте, что все модули корректно импортируются

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

from scipy.sparse import csr_matrix

from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender
from implicit.nearest_neighbours import bm25_weight, tfidf_weight
    
from src.metrics import precision_at_k, recall_at_k
from src.recommenders import MainRecommender

In [2]:
data = pd.read_csv('webinar_4/data/transaction_data.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


Создадим объект класса MainRecommender, в рамках которого производится необходимая префильтрация данных, позволяющая улучшить метрики.

In [3]:
mr = MainRecommender(data=data_train)

/usr/local/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

Посчитаем метрики для наших рекомендаций.

In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(5)

,user_id,actual
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65..."
1,3,"[823704, 834117, 840244, 913785, 917816, 93870..."
2,5,"[913077, 1118028, 1386668]"
3,6,"[825541, 859676, 999318, 1055646, 1067606, 108..."
4,7,"[929248, 948622, 1013572, 1022003, 1049892, 10..."


In [5]:
result['als'] = result['user_id'].apply(lambda x: mr.get_model_recommendations(x))
result.head()

,user_id,actual,als
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65...","[979707, 914190, 961554, 1082185, 995242]"
1,3,"[823704, 834117, 840244, 913785, 917816, 93870...","[908531, 910032, 5568378, 951590, 1053690]"
2,5,"[913077, 1118028, 1386668]","[916122, 874972, 1126899, 5568378, 961554]"
3,6,"[825541, 859676, 999318, 1055646, 1067606, 108...","[903325, 994928, 834484, 904360, 863447]"
4,7,"[929248, 948622, 1013572, 1022003, 1049892, 10...","[893018, 860776, 1127831, 1106523, 1082185]"


In [6]:
result['own'] = result['user_id'].apply(lambda x: mr.get_own_recommendations(x))
result.head()

,user_id,actual,als,own
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65...","[979707, 914190, 961554, 1082185, 995242]","[995242, 1082185, 1029743, 6534178, 1005186]"
1,3,"[823704, 834117, 840244, 913785, 917816, 93870...","[908531, 910032, 5568378, 951590, 1053690]","[1106523, 6534178, 1082185, 951590, 1133018]"
2,5,"[913077, 1118028, 1386668]","[916122, 874972, 1126899, 5568378, 961554]","[6534178, 1126899, 1029743, 1082185, 916122]"
3,6,"[825541, 859676, 999318, 1055646, 1067606, 108...","[903325, 994928, 834484, 904360, 863447]","[1082185, 6534178, 5569230, 1029743, 995242]"
4,7,"[929248, 948622, 1013572, 1022003, 1049892, 10...","[893018, 860776, 1127831, 1106523, 1082185]","[1082185, 6534178, 1106523, 1029743, 1133018]"


In [7]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.227:als
0.2815:own


Рассчитаем рекомендации на основе похожих товаров и юзеров.

In [8]:
result['als_similar_items'] = result['user_id'].apply(lambda x: mr.get_similar_items_recommendation(x))
result.head()

,user_id,actual,als,own,als_similar_items
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65...","[979707, 914190, 961554, 1082185, 995242]","[995242, 1082185, 1029743, 6534178, 1005186]","[1033142, 1071939, 962568, 981760, 1029743]"
1,3,"[823704, 834117, 840244, 913785, 917816, 93870...","[908531, 910032, 5568378, 951590, 1053690]","[1106523, 6534178, 1082185, 951590, 1133018]","[983584, 1131438, 5569327, 883404, 916381]"
2,5,"[913077, 1118028, 1386668]","[916122, 874972, 1126899, 5568378, 961554]","[6534178, 1126899, 1029743, 1082185, 916122]","[985999, 12262778, 1058997, 5569327, 962568]"
3,6,"[825541, 859676, 999318, 1055646, 1067606, 108...","[903325, 994928, 834484, 904360, 863447]","[1082185, 6534178, 5569230, 1029743, 995242]","[1117514, 857849, 970030, 1055646, 948650]"
4,7,"[929248, 948622, 1013572, 1022003, 1049892, 10...","[893018, 860776, 1127831, 1106523, 1082185]","[1082185, 6534178, 1106523, 1029743, 1133018]","[1096036, 834484, 1024306, 1133018, 981760]"


In [9]:
result['als_similar_users'] = result['user_id'].apply(lambda x: mr.get_similar_users_recommendation(x))
result.head()

,user_id,actual,als,own,als_similar_items,als_similar_users
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65...","[979707, 914190, 961554, 1082185, 995242]","[995242, 1082185, 1029743, 6534178, 1005186]","[1033142, 1071939, 962568, 981760, 1029743]","[1029743, 995242, 995242, 1005186, 961554]"
1,3,"[823704, 834117, 840244, 913785, 917816, 93870...","[908531, 910032, 5568378, 951590, 1053690]","[1106523, 6534178, 1082185, 951590, 1133018]","[983584, 1131438, 5569327, 883404, 916381]","[916122, 1133018, 5569230, 995242, 1082185]"
2,5,"[913077, 1118028, 1386668]","[916122, 874972, 1126899, 5568378, 961554]","[6534178, 1126899, 1029743, 1082185, 916122]","[985999, 12262778, 1058997, 5569327, 962568]","[1126899, 1068719, 916122, 1126899, 1126899]"
3,6,"[825541, 859676, 999318, 1055646, 1067606, 108...","[903325, 994928, 834484, 904360, 863447]","[1082185, 6534178, 5569230, 1029743, 995242]","[1117514, 857849, 970030, 1055646, 948650]","[9526410, 1096036, 866211, 5569471, 994928]"
4,7,"[929248, 948622, 1013572, 1022003, 1049892, 10...","[893018, 860776, 1127831, 1106523, 1082185]","[1082185, 6534178, 1106523, 1029743, 1133018]","[1096036, 834484, 1024306, 1133018, 981760]","[854405, 1082185, 866211, 5568378, 866211]"


In [10]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.227:als
0.2815:own
0.1308:als_similar_items
0.1424:als_similar_users


Как видно для этих рекомендаций значения метрик оказались ниже.